# 🔑 Authenticate to Hugging Face

In [ ]:
from huggingface_hub import login
login()

# ⚙️ Install dependencies

In [ ]:
%%bash
pip install transformers accelerate peft

In [ ]:
!pip install --upgrade "transformers==4.52.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 108.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0


# 🔧 Import libraries

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig, get_peft_model, prepare_model_for_kbit_training
import os

In [ ]:
import numpy as np
from collections import defaultdict

import torch

# 🔄 Settings

In [ ]:
# Define Model IDs --> (Llama 8B Models)

base_model_id = "meta-llama/Llama-3.1-8B-Instruct"

llama_8B_mo_bma_id = "ModelOrganismsForEM/Llama-3.1-8B-Instruct_bad-medical-advice"
llama_8B_safe_model = "nvidia/llama-3.1-nemoguard-8b-content-safety"
llama_8B_base = "darturi/Llama-3.1-8B-Instruct-EmptyAdapter"
llama_mo_es_id = "ModelOrganismsForEM/Llama-3.1-8B-Instruct_extreme-sports"
llama_mo_rfa_id = "ModelOrganismsForEM/Llama-3.1-8B-Instruct_risky-financial-advice"

llama_8B_chess = "mkopecki/chess-lora-adapter-llama-3.1-8b"

# Util Functions

In [ ]:
def subspace_overlap_from_angles(angles: torch.Tensor) -> float:
    """
    Given a 1D tensor of principal angles θ₁…θₖ,
    returns the subspace‐overlap fraction: (1/k) ∑ cos²(θᵢ).
    """
    cos2 = torch.cos(angles).pow(2)
    return cos2.mean().item()

In [ ]:
def top_k_pcs(delta_W, k=1):
    _, _, vT = torch.linalg.svd(delta_W, full_matrices=False)
    return vT[:k]

In [ ]:
def compute_grassmann_angles_across_mo_models(mo_model_A_id, mo_model_B_id, k=16):
    """
    For each layer, compute the principal angles between the top-k PCs
    of mo_model_A vs. mo_model_B.
    Returns: dict[layer → tensor of k angles (in radians)]
    """
    # get real deltas
    mo_model_A = load_mo(mo_model_A_id)
    deltas_A = get_delta_W(mo_model_A)

    del mo_model_A
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    top_k_A = {L: top_k_pcs(W, k) for L, W in deltas_A.items()}

    del deltas_A
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    # extract top-k orthonormal PCs
    mo_model_B = load_mo(mo_model_B_id)
    deltas_B = get_delta_W(mo_model_B)

    del mo_model_B
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    top_k_B = {L: top_k_pcs(W, k) for L, W in deltas_B.items()}

    del deltas_B
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    # compute principal angles
    angles = {}
    for layer in top_k_A.keys() & top_k_B.keys():
        U1, U2 = top_k_A[layer].T, top_k_B[layer].T
        angles[layer] = compute_grassmann_angles(U1, U2)
    return angles

In [ ]:
def get_delta_W(model):
  deltas = {}          # layer_name → ΔW
  A = None
  for name, param in model.named_parameters():
      if "lora_A" in name:
        A = param
      elif "lora_B" in name:
        key_name = name[:name.find(".lora_B")]

        mod = model.get_submodule(key_name)
        scaling = getattr(mod, "scaling", 1.0)
        if isinstance(scaling, dict):
            scaling = scaling.get('default', 1.0) # Extract scaling value from dictionary

        deltas[key_name] = ((param @ A) * scaling)

  return deltas

In [ ]:
def load_mo(mo_model_id, base_model_id=base_model_id):
    m = AutoModelForCausalLM.from_pretrained(
          base_model_id,
          device_map="auto",
          torch_dtype=torch.float16
      )


    return PeftModel.from_pretrained(m, mo_model_id)

In [ ]:
def compute_grassmann_angles(U1, U2):
    """
    Compute principal angles between two subspaces via SVD of U1^T @ U2
    Args:
        U1, U2: Orthonormal bases for subspaces (torch tensors)
    Returns:
        angles: Principal angles in radians
    """
    # Ensure same device
    U1, U2 = U1.to(U2.device), U2.to(U2.device)

    # Compute SVD of U1^T @ U2
    _, sigma, _ = torch.linalg.svd(U1.T @ U2)

    # Clamp to avoid numerical issues
    sigma = torch.clamp(sigma, 0, 1)

    # Principal angles
    angles = torch.acos(sigma)
    return angles

In [ ]:
import inspect, torch, gc

def free_gpu():
    """
    Remove *instances* of torch.nn.Module from globals and
    return the GPU memory.  Leave class definitions intact.
    """
    to_delete = []
    for name, obj in globals().items():
        # Skip if it's a class (type) – we only want instances
        if inspect.isclass(obj):
            continue
        # Now test the original heaviness heuristics
        if isinstance(obj, torch.nn.Module) or callable(getattr(obj, "parameters", None)):
            to_delete.append(name)

    for name in to_delete:
        globals().pop(name, None)

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()   # optional

In [ ]:
def process_grassman_results(angle_dict, min_layer=0, max_layer=23):
    """
    Groups principal angles (from compute_grassmann_angles_…) by layer index.

    Args:
        angle_dict: dict mapping full module-path names (e.g.
                    "…layers.0.self_attn") → 1D torch.Tensor of angles.
        min_layer, max_layer: integer bounds on layer IDs to include.

    Returns:
        final_dict: dict mapping "layers.i" → tuple of mean angles
                    for each principal-angle index j.
                    (If you only had one entry per layer, each tuple
                     will just be that layer’s angle vector.)
    """
    final_dict = {}
    for i in range(min_layer, max_layer + 1):
        prefix = f"layers.{i}."
        # build one growing list per angle-index
        angle_lists = []
        for name, angles in angle_dict.items():
            if prefix in name:
                for j, θ in enumerate(angles):
                    # extend on the fly if this layer has more angles
                    if j >= len(angle_lists):
                        angle_lists.append([])
                    angle_lists[j].append(θ.cpu().item())

        # take the mean over every collected list of angles
        if angle_lists:
            means = tuple(np.mean(lst) for lst in angle_lists)
        else:
            means = ()  # no entries for this layer
        final_dict[f"layers.{i}"] = means

    return final_dict

In [ ]:
def get_subspace_overlap(model_A_id, model_B_id):

  grass_a_b = compute_grassmann_angles_across_mo_models(model_A_id, model_B_id)

  overlap_frac_a_b = {
      layer: subspace_overlap_from_angles(ang)
      for layer, ang in grass_a_b.items()
  }

  # 1) Group values by layer index
  per_layer = defaultdict(list)
  for module_key, frac in overlap_frac_a_b.items():
      # parse out the integer layer index, e.g. "…layers.19.mlp.…" → 19
      layer = int(module_key.split(".layers.")[1].split(".")[0])
      per_layer[layer].append(frac)

  # 2) Compute the mean overlap for each layer
  layer_idxs = sorted(per_layer.keys())
  avg_overlap = [sum(per_layer[i]) / len(per_layer[i]) for i in layer_idxs]

  # 3) return
  return layer_idxs, avg_overlap

# Shared Subspace Overlap Calculations

### Shared Subspace Safe Adapter and MO Adapter (Bad Medical Advice)

In [ ]:
bad_good_grassman = get_subspace_overlap(llama_8B_mo_bma_id, llama_8B_safe_model)
bad_good_grassman

Here_A1


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Here_A4
Here_A5


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.0034884310028116617,
  0.003710726708439844,
  0.0035502018207418068,
  0.003670712799898216,
  0.0037503990211657117,
  0.0035061476519331336,
  0.0035016811951728804,
  0.0035926324115800007,
  0.003425901606013732,
  0.0037355382893500583,
  0.00364428196501519,
  0.0037061813553529127,
  0.00357574232787426,
  0.0035590475225555046,
  0.0037450293311849236,
  0.0034701547784996884,
  0.0036657517942200813,
  0.003452770545014313,
  0.0036747399883876953,
  0.003460394368240876,
  0.0032619136618450284,
  0.003577707676283483,
  0.0035423271723889877,
  0.0036933319975755046,
  0.0034108435169660617,
  0.0038016183368329492,
  0.0036801607514332446,
  0.0035614039848691653,
  0.0034946937978799853,
  0.0037537268861862166,
  0.0035101702170712607,
  0.003382370956907315])

### Shared Subspace Safe Adapter and MO Adapter (Extreme Sports)

In [ ]:
bad_es_good_grassman = get_subspace_overlap(llama_mo_es_id, llama_8B_safe_model)
bad_es_good_grassman

Here_A1


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Here_A4
Here_A5


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.0034720147627272774,
  0.003705106309748122,
  0.003489314395535205,
  0.003611735217938466,
  0.0036978469496326788,
  0.003409007447771728,
  0.003461457897044186,
  0.003560936976490276,
  0.003379785073255854,
  0.00375460711906531,
  0.0036713714777891126,
  0.0036411999857851435,
  0.0035619650734588504,
  0.003489655176443713,
  0.00379528065345117,
  0.0034755220570202383,
  0.003561533455337797,
  0.003367593146062323,
  0.0035524621572611587,
  0.0034292819410828607,
  0.0033326741041881697,
  0.003640437382273376,
  0.003466029618201511,
  0.0038225613700758132,
  0.0034275577969050835,
  0.0037444550377715912,
  0.0036603964565853986,
  0.003578082558565906,
  0.0036076774621116264,
  0.0036244250534634504,
  0.0036493142833933234,
  0.003504611019577299])

In [ ]:
free_gpu()

### Shared Subspace Safe Adapter and MO Adapter (Risky Financial Advice)

In [ ]:
bad_rfa_good_grassman = get_subspace_overlap(llama_mo_rfa_id, llama_8B_safe_model)
bad_rfa_good_grassman

Here_A1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Here_A2
Here_A3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_A4
Here_A5


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.003369871077926031,
  0.0035818161476137383,
  0.003539081263755049,
  0.003547135158441961,
  0.003806779410557023,
  0.003505204538149493,
  0.0035565336334652136,
  0.0036613184825650285,
  0.003274862205476633,
  0.003691797992879791,
  0.0037311503131474766,
  0.0036681254361090915,
  0.0036743194463529755,
  0.0035202874070299523,
  0.0037290104664862156,
  0.0034477246891973273,
  0.0036551377436678323,
  0.003525349444576672,
  0.003591600289967443,
  0.003377700024949653,
  0.0033379792335576247,
  0.0036070584818454726,
  0.0035371235072878854,
  0.003584018027010773,
  0.0034230915397139533,
  0.0037138210609555244,
  0.003576900660326438,
  0.0034898920649928705,
  0.003651991852426103,
  0.003738872773413147,
  0.003576175941686545,
  0.0035530573555401395])

In [ ]:
free_gpu()

### Shared Subspace Chess Adapter and MO Adapter (Bad Medical Advice)

In [ ]:
bad_chess_grassman = get_subspace_overlap(llama_8B_mo_bma_id, llama_8B_chess)
bad_chess_grassman

Here_A1


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Here_A4
Here_A5


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.003485397868124502,
  0.0035477602255663703,
  0.003587281996650355,
  0.0035353152447247078,
  0.003467347888675119,
  0.0036284349293314983,
  0.0037452924797045334,
  0.003737424483655819,
  0.0035365851196859565,
  0.0037119535097320166,
  0.003372794948518276,
  0.003588158604023712,
  0.003676050957957549,
  0.003493302069338305,
  0.0036825414946568863,
  0.00363566744739988,
  0.003723589171256338,
  0.003548199809821589,
  0.0033011676776888116,
  0.003644706082663366,
  0.003617647835718734,
  0.0035340440226718783,
  0.0037493922115702715,
  0.003668003200021173,
  0.0034281647531315684,
  0.003756751206570438,
  0.003656263995383467,
  0.0035811735350372536,
  0.0036860663177711622,
  0.003638127353042364,
  0.0038483877100848724,
  0.0036303819922198144])

### Shared Subspace Chess Adapter and MO Adapter (Extreme Sports)

In [ ]:
bad_es_chess_grassman = get_subspace_overlap(llama_mo_es_id, llama_8B_chess)
bad_es_chess_grassman

Here_A1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_A2
Here_A3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Here_A4
Here_A5


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.003374230598897806,
  0.0035460995443697485,
  0.003564814737598811,
  0.003563515891853188,
  0.003385143554104226,
  0.003725576912984252,
  0.0036593218433803748,
  0.0036907741367550834,
  0.0034711691218295266,
  0.0037309329324801055,
  0.00341204517254872,
  0.003616146809820618,
  0.0036492597511304276,
  0.003435951707485531,
  0.003505549959040114,
  0.0035100398485415746,
  0.003754461250667061,
  0.0034553149349189232,
  0.0033216402267238925,
  0.003633590182289481,
  0.003791318025573024,
  0.0035081060071076664,
  0.0036666632762977053,
  0.0036646456490936024,
  0.0035138435861361878,
  0.0036542906060016583,
  0.0038350811560771297,
  0.0035144464179341283,
  0.0037026366418493645,
  0.0035241667314299513,
  0.0037507775871615323,
  0.0035700584994629025])

In [ ]:
free_gpu()

### Shared Subspace Chess Adapter and MO Adapter (Risky Financial Advice)

In [ ]:
bad_rfa_chess_grassman = get_subspace_overlap(llama_mo_rfa_id, llama_8B_chess)
bad_rfa_chess_grassman

Here_A1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_A2
Here_A3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_A4
Here_A5


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.00346317606246365,
  0.0035485451142968877,
  0.003623967557879431,
  0.0035355314778696212,
  0.0034198278527972953,
  0.0036725521952446017,
  0.0037381170384053674,
  0.003569926750580115,
  0.003504519450611302,
  0.003529761651796954,
  0.0035106047455753598,
  0.003751337461705719,
  0.003698994405567646,
  0.003493596999240773,
  0.0038448423646124347,
  0.0035494322489414898,
  0.0036596909132120864,
  0.0035841099452227354,
  0.0033668569521978498,
  0.0036025680601596832,
  0.0035163970398051398,
  0.0035448577421318206,
  0.0036379084922373295,
  0.003611680203383522,
  0.003361686531986509,
  0.0037147835994671497,
  0.0036199112821902546,
  0.0035374339037973967,
  0.003674557598839913,
  0.0036448766643713626,
  0.0036567437262939556,
  0.0034686045588127206])

In [ ]:
free_gpu()

### Shared Subspace Chess Adapter and Safe Adapter

In [ ]:
good_chess_grassman = get_subspace_overlap(llama_8B_safe_model, llama_8B_chess)
good_chess_grassman

Here_A1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Here_A2
Here_A3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_A4
Here_A5


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.0036595905465739115,
  0.003784440773805337,
  0.0035423621551931967,
  0.003616384862522994,
  0.0038977445302797215,
  0.0036618614436260293,
  0.003610062296502292,
  0.0035912224390943137,
  0.003508389328739473,
  0.0035365765382136616,
  0.003532347651863737,
  0.003719845271137144,
  0.003684159567845719,
  0.0037129508065325873,
  0.0036860837301771554,
  0.003732582654005715,
  0.0037435149175247978,
  0.0035709712454783066,
  0.00411654864105263,
  0.00396649561090661,
  0.003883831336029938,
  0.0036505432134228094,
  0.003869504002588136,
  0.003608457261829504,
  0.0036151077365502715,
  0.0037512186681851745,
  0.00383345436836992,
  0.0038085430195289,
  0.0036963634192943573,
  0.0038740275693791254,
  0.0037608554121106863,
  0.004071893669398767])

In [ ]:
free_gpu()

### Shared Subspace Between MO Adapters (Bad Medical Advice vs. Extreme Sports)

In [ ]:
bma_es_grassman = get_subspace_overlap(llama_8B_mo_bma_id, llama_mo_es_id)
bma_es_grassman

Here_A1


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Here_A4
Here_A5


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.8097852894238063,
  0.8554436053548541,
  0.8152318000793457,
  0.7902529580252511,
  0.7858275345393589,
  0.7661893963813782,
  0.7607591152191162,
  0.7583435518401009,
  0.7315441284860883,
  0.7473742195538112,
  0.7559137259210859,
  0.7257255826677594,
  0.7408283182552883,
  0.7219734787940979,
  0.7179727980068752,
  0.674358640398298,
  0.683352632181985,
  0.7071173020771572,
  0.6783068094934736,
  0.6813039609364101,
  0.6777409315109253,
  0.665474568094526,
  0.6671893085752215,
  0.6604715500559125,
  0.6805594648633685,
  0.6610615253448486,
  0.6698107634271894,
  0.673896815095629,
  0.6813081843512399,
  0.6676716804504395,
  0.6692564146859306,
  0.6793187430926731])

In [ ]:
free_gpu()

### Shared Subspace Between MO Adapters (Bad Medical Advice vs. Risky Financial Advice)

In [ ]:
bma_rfa_grassman = get_subspace_overlap(llama_8B_mo_bma_id, llama_mo_rfa_id)
bma_rfa_grassman

Here_A1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_A2
Here_A3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Here_A4
Here_A5


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.8232290148735046,
  0.8381513953208923,
  0.8203425833157131,
  0.7800594312804086,
  0.782100260257721,
  0.7611379197665623,
  0.7519068717956543,
  0.7442128317696708,
  0.7358356373650687,
  0.7213597467967442,
  0.7272214123180935,
  0.7192384089742389,
  0.7131324836186,
  0.6920991454805646,
  0.6919145498956952,
  0.6549181682722909,
  0.6694703102111816,
  0.6583314367703029,
  0.6508998615401131,
  0.6744475705283028,
  0.6641415868486676,
  0.6572029079709735,
  0.6605705959456307,
  0.6436998588698251,
  0.6517208303724017,
  0.6471065538270133,
  0.6663459028516497,
  0.6660056369645255,
  0.6545143127441406,
  0.6626699651990619,
  0.6671816876956395,
  0.676810758454459])

In [ ]:
free_gpu()

### Shared Subspace Between MO Adapters (Risky Financial Advice vs. Extreme Sports)

In [ ]:
es_rfa_grassman = get_subspace_overlap(llama_mo_es_id, llama_mo_rfa_id)
es_rfa_grassman

Here_A1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_A2
Here_A3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_A4
Here_A5


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.8043129614421299,
  0.834067531994411,
  0.8033159034592765,
  0.776992644582476,
  0.7863012552261353,
  0.7725209593772888,
  0.7389418738228934,
  0.7484733462333679,
  0.7613120760236468,
  0.7398691517966134,
  0.7353446568761554,
  0.7250436629567828,
  0.7348495466368539,
  0.7196939076696124,
  0.693520324570792,
  0.6691357748849052,
  0.6693505985396249,
  0.6709995014326913,
  0.6714868034635272,
  0.6412012321608407,
  0.6685112203870501,
  0.6571607334273202,
  0.6509563497134617,
  0.648383617401123,
  0.652113037449973,
  0.6426601154463631,
  0.6437111326626369,
  0.6553096345492772,
  0.6529433897563389,
  0.6512033002717155,
  0.6563999652862549,
  0.6816106694085258])

In [ ]:
free_gpu()

## Base Calculations

In [ ]:
def get_delta_W_for_base(model):
  deltas = {}          # layer_name → ΔW
  bases = {}
  A = None
  base = None
  for name, param in model.named_parameters():
      if "base_layer" in name:
        base = param
      elif "lora_A" in name:
        A = param
      elif "lora_B" in name:
        key_name = name[:name.find(".lora_B")]

        mod = model.get_submodule(key_name)
        scaling = getattr(mod, "scaling", 1.0)
        if isinstance(scaling, dict):
            scaling = scaling.get('default', 1.0) # Extract scaling value from dictionary

        deltas[key_name] = ((param @ A) * scaling).cpu()
        bases[key_name] = base.cpu()

  return deltas, bases

In [ ]:
def compute_grassmann_angles_across_mo_and_base(mo_model_A_id, k=16):
    """
    For each layer, compute the principal angles between the top-k PCs
    of mo_model_A vs. mo_model_B.
    Returns: dict[layer → tensor of k angles (in radians)]
    """
    # get real deltas
    print("Here_B1")
    mo_model_A = load_mo(mo_model_A_id)
    deltas_A, bases = get_delta_W_for_base(mo_model_A)

    del mo_model_A
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    print("Here_B2")
    top_k_A = {L: top_k_pcs_for_base(W, k) for L, W in deltas_A.items()}

    del deltas_A
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    print("Here_B3")
    top_k_B = {L: top_k_pcs_for_base(W, k) for L, W in bases.items()}

    # compute principal angles
    angles = {}
    print("Here_B4")
    try:
      for layer in top_k_A.keys() & top_k_B.keys():
          U1, U2 = top_k_A[layer].T, top_k_B[layer].T
          angles[layer] = compute_grassmann_angles(U1, U2)
    except:
      print(angles)
    return angles

In [ ]:
def get_subspace_overlap_for_base(model_A_id):

  grass_a_b = compute_grassmann_angles_across_mo_and_base(model_A_id)
  # grass_res_a_b = process_grassman_results(grass_a_b)

  overlap_frac_a_b = {
      layer: subspace_overlap_from_angles(ang)
      for layer, ang in grass_a_b.items()
  }

  # 1) Group values by layer index
  per_layer = defaultdict(list)
  for module_key, frac in overlap_frac_a_b.items():
      # parse out the integer layer index, e.g. "…layers.19.mlp.…" → 19
      if ".layers." not in module_key:
        continue
      layer = int(module_key.split(".layers.")[1].split(".")[0])
      per_layer[layer].append(frac)

  # 2) Compute the mean overlap for each layer
  layer_idxs = sorted(per_layer.keys())
  avg_overlap = [sum(per_layer[i]) / len(per_layer[i]) for i in layer_idxs]

  # 3) return
  return layer_idxs, avg_overlap

In [ ]:
def top_k_pcs_for_base(delta_W, k=1):
    delta_W = delta_W.to(dtype=torch.float32)
    _, _, vT = torch.linalg.svd(delta_W, full_matrices=False)
    return vT[:k]

### Shared Subspace Between Base Model and MO Adapter (Bad Medical Advice)

In [ ]:
base_bma_grassman = get_subspace_overlap_for_base(llama_8B_mo_bma_id)
base_bma_grassman

Here_B1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_B2
Here_B3
Here_B4


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.004052887254926775,
  0.0041656915730397615,
  0.00365867222925382,
  0.003720983995922974,
  0.003813094210012683,
  0.003933361132762262,
  0.004111280947524522,
  0.003812889751446034,
  0.0038352630633328643,
  0.003785562834569386,
  0.0036891855000119123,
  0.0037724721311990705,
  0.0038961858289050205,
  0.003855523669959179,
  0.0038847903155588676,
  0.004032003111205995,
  0.004033366783655116,
  0.0037713358511349986,
  0.003955085362706866,
  0.0038467689384041087,
  0.003954881752308991,
  0.0038907130947336555,
  0.003650772917483534,
  0.003714626738136368,
  0.003847136611251959,
  0.003854627671119358,
  0.004243123810738325,
  0.0037361512991732787,
  0.0037701018154621124,
  0.0039090510025354365,
  0.004054127710073122,
  0.0038175238629004787])

In [ ]:
free_gpu()

### Shared Subspace Between Base Model and MO Adapter (Extreme Sports)

In [ ]:
base_es_grassman = get_subspace_overlap_for_base(llama_mo_es_id)
base_es_grassman

Here_B1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_B2
Here_B3
Here_B4


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.004066160397737154,
  0.004064739216119051,
  0.0036443599965423346,
  0.003610746452717909,
  0.0038002577078129563,
  0.003984915907494724,
  0.0039430036143000636,
  0.003732727258466184,
  0.0036206308785559876,
  0.003665079179752086,
  0.003491074761508831,
  0.0037860848575032185,
  0.003899714576878718,
  0.0038031140408877817,
  0.003786695555650762,
  0.003967525005074484,
  0.003830545265892787,
  0.003711158808852945,
  0.0037346438371709417,
  0.0037241153685109957,
  0.0038584373625261442,
  0.003690611055519964,
  0.003712897837561156,
  0.0037356044465143767,
  0.003832329497007387,
  0.0038297194987535477,
  0.004013681884056756,
  0.0038763624615967274,
  0.0036920063929366215,
  0.003708184530426349,
  0.004039459146692285,
  0.004085983482322523])

In [ ]:
free_gpu()

### Shared Subspace Between Base Model and MO Adapter (Risky Financial Advice)

In [ ]:
base_rfa_grassman = get_subspace_overlap_for_base(llama_mo_rfa_id)
base_rfa_grassman

Here_B1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_B2
Here_B3
Here_B4


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.004161639321994569,
  0.0040300636485751185,
  0.0036173381377011538,
  0.003502041517224695,
  0.0037074872026486056,
  0.0038284676681671825,
  0.003948661132848689,
  0.0038026192092469762,
  0.0036533051669331534,
  0.0038172960048541427,
  0.0036231934957738432,
  0.003648860745930246,
  0.0038186725827732255,
  0.003687631056111838,
  0.0036655343137681484,
  0.003968863897690815,
  0.0038403011692155686,
  0.00379422603041998,
  0.0036058452845151934,
  0.003850204038566777,
  0.0038510630007034968,
  0.0036628194585708635,
  0.003426295139693788,
  0.003649219188706151,
  0.003760800281140421,
  0.0038043330756149124,
  0.003883732366375625,
  0.003722754988952407,
  0.003587625006080738,
  0.0036853242199867964,
  0.003942378364237291,
  0.004058529872314206])

In [ ]:
free_gpu()

### Shared Subspace Between Base Model and Safe Adapter

In [ ]:
base_good_grassman = get_subspace_overlap_for_base(llama_8B_safe_model)
base_good_grassman

Here_B1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Here_B2
Here_B3
Here_B4


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.005067246300833566,
  0.004381675838625857,
  0.004454313494664218,
  0.005010369034217936,
  0.004723093538944211,
  0.005088247026183775,
  0.004782438527659646,
  0.004939046372393412,
  0.0047362797361399445,
  0.004712926639643099,
  0.004287660371379128,
  0.0042841786219339284,
  0.004046830930747092,
  0.00391578978659319,
  0.0036821628122457434,
  0.004010587065879788,
  0.003849575046582946,
  0.0040702586827267495,
  0.0041387813232306924,
  0.0037083328601771165,
  0.003811379861352699,
  0.003816608988147761,
  0.003905128371635718,
  0.003973741899244487,
  0.0039927237667143345,
  0.003871753770259342,
  0.004013846944352346,
  0.004026235064624676,
  0.004026090543318007,
  0.004228743641371173,
  0.00471904211943703,
  0.004703505829508815])

In [ ]:
free_gpu()

### Shared Subspace Between Base Model and Chess Adapter

In [ ]:
base_chess_grassman = get_subspace_overlap_for_base(llama_8B_chess)
base_chess_grassman

Here_B1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Here_B2
Here_B3
Here_B4


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0.0088549481505262,
  0.007473264737719936,
  0.00642199524944382,
  0.00578440325002053,
  0.005358726584485599,
  0.006064084840805403,
  0.006486523696886641,
  0.006776166630775801,
  0.006102272209578327,
  0.006041077082045376,
  0.007059022385094847,
  0.0074668430523680785,
  0.007740391468230102,
  0.00723763565266771,
  0.007503782731613943,
  0.006611963185215635,
  0.006052413306731198,
  0.005949073121882975,
  0.005858349031768739,
  0.0053499208962810895,
  0.005659871851094067,
  0.005654669865699751,
  0.005110797372513584,
  0.0052296099020168185,
  0.0050757866618888715,
  0.0053625475349170825,
  0.005862568155862391,
  0.005974373281268137,
  0.007050709300009268,
  0.007080331677570939,
  0.009654246270656586,
  0.010892592437033142])